### Survey of Consumer Finances 



In [14]:
%config Completer.use_jedi = False
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

from io import BytesIO, TextIOWrapper, StringIO
from zipfile import ZipFile

# Format values as dollars
numfmt = (lambda x: '\${:,.0f}'.format(x) if x >=0 
          else '-\${:,.0f}'.format(abs(x)))

In [33]:
# Download summary zip file and save selected parts
url = 'https://www.federalreserve.gov/econres/scf/dataviz/download/zips/scf.zip'
r = requests.get(url)
zip_file = ZipFile(BytesIO(r.content))
for i in ['racecl4_mean', 'racecl4_median', 'racecl4_have', 'nwcat_mean', 
          'nwcat_median']:
    with zip_file.open(f'interactive_bulletin_charts_{i}.csv', 'r') as csvfile:
        (pd.read_csv(StringIO(csvfile.read().decode()), 
                     index_col=['year', 'Category'], 
                     parse_dates=True)
         .to_csv(data_dir / f'{i}.csv'))

#### Wealth and Income

In [165]:
# Store data for pre-tax income by wealth percentile chart
df = (pd.read_csv(data_dir / 'nwcat_median.csv', 
                  index_col=['year', 'Category'], parse_dates=True)
      ['Before_Tax_Income'].unstack())
df.to_csv(data_dir / 'scfwealthmedian.csv', index_label='date')

# End nodes
c = {'Less than 25': 'violet',
     '25-49.9': 'red!20!orange!95!black',
     '50-74.9': 'green!80!blue!85!black', 
     '75-89.9': 'red!80!black',
     '90-100': 'blue!80!cyan!80!black'}

date = {k: None for k, v in c.items()}
date['90-100'] = 'y'

nodes = '\n'.join([end_node(df[k], v, date=date[k], full_year=True) 
                   for k, v in c.items()])
write_txt(text_dir / 'scfwealthmedian_nodes.txt', nodes)

# Text
yr = dtxt(df.index[-1])['year']
write_txt(text_dir / 'scf_year.txt', yr)
pryr = dtxt(df.index[0])['year']
yrch = str(int(yr) - int(pryr))

d, t = {}, {}
for cat in ['median', 'mean']:
    s = (pd.read_csv(data_dir / f'nwcat_{cat}.csv', 
                     index_col=['year', 'Category'], parse_dates=True)
         ['Net_Worth'].unstack())
    d[cat] = s
    t[cat] = s.multiply(1000).round(-2).applymap(numfmt).iloc[-1].to_dict()
    
inc = df.multiply(1000).round(-2).applymap(numfmt)
lti = inc.iloc[-1]
pyi = inc.iloc[0]
chlt = (df.diff(len(df)-1).iloc[-1].multiply(1000)
          .round(-2).apply(numfmt))
pct = cagr(df)

text1 = ('The top ten percent of families by wealth, percentiles 90 '+
         f'to 100 with a mean wealth of \${p90mean:.1f} million and '+
         f'a median wealth of \${p90median:.1f} million in {yr}, have '+
         f'a typical annual income of {lti["90-100"]} in {yr} and '+
         f'{pyi["90-100"]} in {pryr}, after adjusting for inflation. '+
         f'Income for the group increased {chlt["90-100"]}, or '+
         f'{pct["90-100"]:.1f} percent per year, over the {yrch}-year '+
         'period.')
write_txt(text_dir / 'income_wealthg1.txt', text1)
print(text1)

text2 = ('Families in the third wealth quartile (50th to 74.9th '+
         f'percentiles, mean wealth of {t["mean"]["50-74.9"]} in {yr}), '+
         f'have a typical income of {lti["50-74.9"]} in {yr} and '+
         f'{pyi["50-74.9"]} in {pryr} {cl["50-74.9"]}, an increase of '+
         f'{chlt["50-74.9"]} ({pct["50-74.9"]:.1f} percent per year).\n\n'+
         'Second quartile (25th to 49.9th percentile, mean wealth of '+
         f'{t["mean"]["25-49.9"]}) family income increased '+
         f'{chlt["25-49.9"]} ({pct["25-49.9"]:.1f} percent per year) to '+
         f'{lti["25-49.9"]} in {yr}, from {pyi["25-49.9"]} in {pryr} '+
         f'{cl["25-49.9"]}.\n\n For the bottom quarter of families '+
         f'by wealth {cl["Less than 25"]}, typical income increased '+
         f'{chlt["Less than 25"]} or {pct["Less than 25"]:.1f} percent '+
         f'per year to {lti["Less than 25"]}, over the {yrch} years '+
         f'ending {yr}. The bottom quarter of families have no wealth.')
write_txt(text_dir / 'income_wealthg2.txt', text2)
print('\n', text2)

The top ten percent of families by wealth, percentiles 90 to 100 with a mean wealth of \$7.8 million and a median wealth of \$3.8 million in 2022, have a typical annual income of \$301,600 in 2022 and \$180,400 in 1989, after adjusting for inflation. Income for the group increased \$121,200, or 20.5 percent per year, over the 33-year period.

 Families in the third wealth quartile (50th to 74.9th percentiles, mean wealth of \$373,800 in 2022), have a typical income of \$83,200 in 2022 and \$72,200 in 1989 (see {\color{green!80!blue!85!black}\textbf{---}}), an increase of \$11,100 (5.3 percent per year).

Second quartile (25th to 49.9th percentile, mean wealth of \$98,800) family income increased \$8,900 (6.1 percent per year) to \$59,500 in 2022, from \$50,500 in 1989 (see {\color{red!20!orange!95!black}\textbf{---}}).

 For the bottom quarter of families by wealth (see {\color{violet}\textbf{---}}), typical income increased \$10,500 or 14.1 percent per year to \$34,600, over the 33 ye

#### Wealth and Race

In [163]:
# Collect data
d = {'Black, non-Hispanic': 'Black', 
     'White, non-Hispanic': 'White'}

# Wealth, Income, Asset measures
srs = {'Net_Worth': 'scfracemean.csv', 
       'Before_Tax_Income': 'scfracemean1.csv',
       'Financial_Assets': 'scfracemean0.csv', 
       'Stock_Holdings': 'scfracemean2.csv'}
for name, filename in srs.items():
    df = (pd.read_csv(data_dir / 'racecl4_mean.csv', 
                      index_col=['year', 'Category'], 
                      parse_dates=True)
          [name].unstack().rename(d, axis=1))
    df.to_csv(data_dir / filename, index_label='date')
wval = df.White.iloc[-1] * 1000
bval = df.Black.iloc[-1] * 1000

# Share that own stocks
df = (pd.read_csv(data_dir / 'racecl4_have.csv', 
                  index_col=['year', 'Category'], 
                  parse_dates=True)
      ['Stock_Holdings'].unstack().rename(d, axis=1))
df.to_csv(data_dir / 'scfracehave.csv', index_label='date')

# Median wealth
df = (pd.read_csv(data_dir / 'racecl4_median.csv', 
                  index_col=['year', 'Category'], 
                  parse_dates=True)
      ['Net_Worth'].unstack().rename(d, axis=1))
df.to_csv(data_dir / 'scfracemedian.csv', index_label='date')

#Text
wp = df.White.iloc[-1]
bp = df.Black.iloc[-1]
yr = dtxt(df.index[-1])['year']
text = (f'In {yr}, among the {wp:.1f} percent of white families '+
        'who own stocks, the average value of stock holdings is '+
        f'\${wval:,.0f}. The return on these assets is a supplement '+
        'to labor income and the assets themselves provide cushion '+
        'against unexpected expenses. Meanwhile, black families '+
        f'have relatively few financial assets; only {bp:.1f} '+
        'percent of black families own stocks, with average stock '+
        f'holdings of \${bval:,.0f}.')
write_txt(text_dir / 'scfracestocks.txt', text)
print(text, '\n')

# Summary text for racial wealth gap
mu = pd.read_csv(data_dir / 'scfracemean.csv', index_col='date', 
                 parse_dates=True)
md = pd.read_csv(data_dir / 'scfracemedian.csv', index_col='date', 
                 parse_dates=True)
tmu = mu.multiply(1000).round(-2).applymap(numfmt).iloc[-1]

text = (f"In {yr}, white non-Hispanic families' average net worth "+
        f'was {tmu.White}, compared to {tmu.Black} for black '+
        f'non-Hispanic families, and {tmu.Hispanic} for Hispanic '+
        'families of any race. Additionally, typical (median) family '+
        'wealth is much lower than average (mean) family wealth, '+
        'as the result of a concentration of wealth among the '+
        'wealthiest families.')
write_txt(text_dir / 'scfracialwealthgap.txt', text)
print(text)

# Racial wealth gap bar chart
ltmu = mu.iloc[-1]
ltmu.index.name = 'name'
ltmu.name = 'value'
order = ['White', 'Black', 'Hispanic', 'Other']
(ltmu[order].to_csv(data_dir / 'race_wealth_gap_mu.csv', 
         index_label='name'))

ltmd = md.iloc[-1]
ltmd.index.name = 'name'
ltmd.name = 'value'
(ltmd[order].to_csv(data_dir / 'race_wealth_gap_md.csv', 
         index_label='name'))

In 2022, among the 284.3 percent of white families who own stocks, the average value of stock holdings is \$568,136. The return on these assets is a supplement to labor income and the assets themselves provide cushion against unexpected expenses. Meanwhile, black families have relatively few financial assets; only 44.1 percent of black families own stocks, with average stock holdings of \$80,400. 

In 2022, white non-Hispanic families' average net worth was \$1,361,800, compared to \$211,600 for black non-Hispanic families, and \$227,500 for Hispanic families of any race. Additionally, typical (median) family wealth is much lower than average (mean) family wealth, as the result of a concentration of wealth among the wealthiest families.


### Wealth Distribution

In [169]:
url = '/home/brian/Documents/econ_data/micro/results/wealth_dist.csv'
df = pd.read_csv(url, index_col='pct')
df.to_csv(data_dir / 'wealth_dist.csv', index_label='pct')

In [179]:
url = 'https://www.federalreserve.gov/econres/scfindex.htm'
sh = df.Share.iloc[-1]
cb = c_box('blue!70!cyan!80!white')
text = ('The vast majority of this wealth, however, is held by the wealthiest '+
        'families. In the 2022 \href{{{url}}}{{Survey of Consumer Finances}}, '+
        f'{sh:.1f} percent of wealth is held by the wealthiest ten percent of '+
        f'families {cb}.')
write_txt(text_dir / 'hh_avg_pc_wealth_summary2.txt', text)
print(text)

The vast majority of this wealth, however, is held by the wealthiest families. In the 2022 \href{{{url}}}{{Survey of Consumer Finances}}, 73.4 percent of wealth is held by the wealthiest ten percent of families (see\cbox{blue!70!cyan!80!white}).
